# Diagnosis System version 2 -- test
This is a testing version for our diagnosis system. Virtual patients are generated and diagnosed with the system, and the accuracy is calculated.

In [21]:
import pandas as pd
import numpy as np

WM = pd.read_csv('../WeightMatrix/Dis_Sym_30.csv', index_col=0)

dis2sym = pd.read_csv('../UMLS/dis_symptom.csv', header=None)

dis2sym.fillna(method='ffill',inplace=True)

umls_dis = {}
umls_sym = {}
dis_num = {}
for i in dis2sym.index:
    temp = dis2sym.loc[i][0]
    items = temp.split('^')
    item = items[0].strip('UMLS:').split('_')
    if len(item) != 2: continue
    umls_dis[item[0]] = item[1]
    dis_num[item[0]] = int(dis2sym.loc[i][1])
for i in dis2sym.index:
    temp = dis2sym.loc[i][2]
    items = temp.split('^')
    item = items[0].strip('UMLS:').split('_')
    if len(item) != 2: continue
    umls_sym[item[0]] = item[1]
    
rev_sym = {v: k for k, v in umls_sym.items()}
rev_dis = {v: k for k, v in umls_dis.items()}

In [22]:
patients = {}

for i in WM.index:
    patients[i] = []
    for j in WM.columns:
        if WM.loc[i][j] != 0:
            patients[i].append(j)
#patients

In [23]:
#patients

In [24]:
def SelectedMatrix(sym):
    selected = WM[WM[sym] != 0]
    selected = selected.drop(columns=[sym])
    for c in selected.columns:
        if sum(selected[c]) == 0:
            selected.drop(columns=[c],inplace=True)
    return selected
    

In [86]:
T = 0.03
def renorm(dia):
    #for c in dia.index:
    #    dia[c] *= dis_num[c]**(1/3)
    dia.sort_values(ascending=False, inplace=True)
    dia.reset_index(drop=True)
    s = sum([np.exp(ai/T) for ai in dia])
    return np.exp(dia/T)/s

In [ ]:
# TEST
T=0.2
import numpy as np
import pandas as pd
a = pd.Series([0.1,0.2,0.3,0.7])
a.sort_values(ascending=False, inplace=True)
s = sum([np.exp(ai/T) for ai in a])
[ai for ai in a]
a = np.exp(a/T)/s
a.reset_index(drop=True)

In [87]:
def diagnosis(dis, sym):
        
    selected = SelectedMatrix(sym)
    
    #The response vector
    res = pd.Series(index=WM.columns, data=[0]*len(WM.columns))
    res[sym] = 1
    
    #Diagnosis process
    output = [0, 0, 0, 0] # result: accuracy, number of questions asked, confidance, top 5 confidence
    while True:
        dia = WM.dot(res)
        if len(selected) == 1:
            #dia[selected.index[0]] = 1
            #dia.sort_values(ascending=False, inplace=True)
            dia = renorm(dia)
            output[2] = dia.iloc[0]
            output[3] = sum(dia.iloc[:5])
            if dia.keys()[0] == dis:
                output[0] = 1
                return output
            else:
                output[0] = -1
                return output
        elif len(selected.columns) == 1:
            output[0] = 0
            return output
        
        output[1] += 1
        #choose the most relevant symptom to ask: The symptom that are least shared with other diseases
        next_i = selected.columns[0]
        s = 100       
        
        for i in selected.columns:
            if min(selected[i]) != 0 or max(selected[i])==0:
                selected.drop(columns=[i], inplace = True)
        
        for i in selected.columns:   
            if 0 in selected[i].value_counts():
                pri = abs(selected[i].value_counts()[0] - len(selected)/2)
                if pri < s:
                    s = pri
                    next_i = i      
            else:
                res[next_i] = 1
                selected = selected[selected[next_i]!=0]  
        
        if next_i in patients[dis]:
            res[next_i] = 1
            selected = selected[selected[next_i]!=0]
        else:
            res[next_i] = 0
            selected = selected[selected[next_i]==0]
            
        selected.drop(columns=[next_i], inplace = True)
    

In [90]:
diagnosis('C0020538', 'C0438696')

[-1, 4, 0.35680643243075755, 0.8003992894545047]

In [45]:
patients.items()

dict_items([('C0020538', ['C0008031', 'C0392680', 'C0012833', 'C0004093', 'C0085639', 'C0039070', 'C0042571', 'C0038990', 'C0030252', 'C0027497', 'C0002962', 'C0438716']), ('C0011847', ['C0008031', 'C0392680', 'C0004093', 'C0042571', 'C0038990', 'C0027497', 'C0032617', 'C0085602', 'C0085619', 'C0034642', 'C0241526', 'C0856054', 'C0042963', 'C0553668']), ('C0011570', ['C0424000', 'C0438696', 'C0233762', 'C0150041', 'C0424109', 'C0917801', 'C0424230', 'C0022107', 'C0312422', 'C0344315', 'C0233763', 'C0233481', 'C0085631', 'C0040822', 'C0728899', 'C0424068', 'C0455769', 'C1299586', 'C0028084', 'C0235198', 'C0237154']), ('C0010054', ['C0008031', 'C0392680', 'C0038990', 'C0002962', 'C0438716', 'C0085619', 'C0086439', 'C0231807', 'C0232292']), ('C0032285', ['C0392680', 'C0034642', 'C0010200', 'C0015967', 'C0029053', 'C0239134', 'C0008033', 'C0457096', 'C0238844', 'C0085593', 'C0035508', 'C0457097', 'C0850149', 'C0043144', 'C0019079', 'C0476273', 'C0231835', 'C0231218', 'C0028081']), ('C00188

In [42]:
def performance():
    perf = pd.DataFrame(columns = ['result' ,'number of question', 'confidence', 'T5 confidence'])
    N = 0
    for d, v in patients.items():
        for s in v:
            N += 1
            perf.loc[N] = diagnosis(d,s)
            #if N%100 == 0: print(N)
    perf.to_csv('testlog.csv')

In [43]:
%%time
import time
performance()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
CPU times: user 1h 20min 36s, sys: 2h 21min 33s, total: 3h 42min 9s
Wall time: 36min


In [91]:
import pandas as pd
a = pd.read_csv('test/testlog.csv')
b = pd.read_csv('test/testlog2.csv')

In [92]:
a.describe()

,Unnamed: 0,result,number of question,confidence,T5 confidence
count,1854.000000,1854.000000,1854.000000,1854.000000,1854.000000
mean,927.500000,-0.355987,3.000539,0.477523,0.720169
std,535.348018,0.934743,1.505701,0.298961,0.291565
min,1.000000,-1.000000,0.000000,0.007522,0.037597
25%,464.250000,-1.000000,2.000000,0.222848,0.561340
50%,927.500000,-1.000000,3.000000,0.448422,0.825419
75%,1390.750000,1.000000,4.000000,0.718122,0.949933
max,1854.000000,1.000000,8.000000,1.000000,1.000000


In [77]:
b.describe()

,Unnamed: 0,result,number of question,confidence,T5 confidence
count,1854.000000,1854.0,1854.000000,1854.000000,1854.000000
mean,927.500000,1.0,3.000539,0.821611,0.839346
std,535.348018,0.0,1.505701,0.035298,0.021941
min,1.000000,1.0,0.000000,0.519379,0.744534
25%,464.250000,1.0,2.000000,0.805974,0.827628
50%,927.500000,1.0,3.000000,0.833213,0.847383
75%,1390.750000,1.0,4.000000,0.847793,0.857429
max,1854.000000,1.0,8.000000,0.856173,0.862244


In [48]:
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 684
Wrong: 1170
fail: 0


In [43]:
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

total: 1854
Correct: 1854
Wrong: 0
fail: 0


In [ ]:
#cube, 0.8
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

In [ ]:
#cube, 0.5
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

In [ ]:
#square, 0.5
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

In [ ]:
#quad, 0.8
N = 0
t = 0
f = 0
for d, v in patients.items():
    for s in v:
        N += 1
        if diagnosis(d,s) == 1:
            t += 1
        if diagnosis(d,s) == 0:
            f += 1
print('total:', N)
print('Correct:', t)
print('Wrong:', N-t-f)
print('fail:', f)

In [46]:
1041/1854

0.5614886731391586

In [12]:
import pandas as pd
a = pd.DataFrame(index = [1,2], columns = ['a','b', 'c', 'd'], data = [[1,2,3,4],[1,2,3,4]])
b = pd.DataFrame(index = [1], columns = ['a'], data = [2])

In [14]:
for i in b.index:
    for j in b.columns:
        if b.loc[i][j]:
            a.loc[i][j] += b.loc[i,j]

In [16]:
a.loc[3]['a'] = 3

KeyError: 'the label [3] is not in the [index]'